In [1]:
import pandas as pd
from bulletin.commom.utils import auto_fit_columns
import codecs

pd.set_option('display.max_columns', None)

information_schema = pd.read_csv('information_schema.csv',usecols=['table_name','column_name','udt_name'])
information_schema

,table_name,column_name,udt_name
0,redcap,dt_obito,float8
1,redcap,idade,float8
2,redcap,fxetaria,float8
3,redcap,exposicao_unidade_saude,float8
4,redcap,nacionalidade,float8
...,...,...,...
2117,sivep_srag,febre,varchar
2118,sivep_srag,tosse,varchar
2119,sivep_srag,garganta,varchar
2120,sivep_srag,dispneia,varchar


In [2]:
notificacao = information_schema.loc[information_schema['table_name']=='notificacao']
notificacao.to_excel('notificacao.xlsx')

In [3]:
with codecs.open('tables.dbml',"w","utf-8-sig") as out:
    notificacao['ref'] = ""
    notificacao.loc[notificacao['column_name']=='id','ref'] = f"[pk]"

    for table_name, table in information_schema.groupby('table_name'):
        if table_name in notificacao['column_name'].values:
            notificacao.loc[notificacao['column_name']==table_name,'ref'] = f"[ref: - {table_name}.{table['column_name'].iloc[0]}]"

            out.write(f"Table {table_name}" + " {\n")
            out.write(f"\t {table['column_name'].iloc[0]} {table['udt_name'].iloc[0]} [pk]\n")
            for column_name, udt_name in zip(table['column_name'][1:],table['udt_name'][1:]):
                out.write(f"\t {column_name} {udt_name}\n")
            out.write("}\n\n")

    out.write(f"Table notificacao" + " {\n")
    for column_name, udt_name,ref in zip(notificacao['column_name'],notificacao['udt_name'],notificacao['ref']):
        out.write(f"\t {column_name} {udt_name} {ref if ref != None else ''}\n")
    out.write("}\n\n")
